In [27]:
import os
import sys
import pandas as pd
import numpy as np

In [32]:
output5 = "./step1/output5"
GTextract = "./step1/output5_GT.tsv"
synonymous = "./step2/list_synonymous.tsv"

!/modules/EasyBuild/software/PLINK/1.9b-6.10/plink  --bfile {output5} --recodeA  --out {GTextract}  --extract {synonymous}


GTextract_name = "./step1/output5_GT.tsv.raw"
print("reading the file : ", GTextract_name)

df_vcf=pd.read_csv(GTextract_name, sep="\s")
print("Reading the output5_GT.tsv.raw")
print(df_vcf.head())

# Drop specified columns
df_vcf = df_vcf.drop(columns=['IID', 'PAT', 'MAT', 'SEX', 'PHENOTYPE'])

# Compute the number of synonymous mutations
df_vcf["count_synonymous"] = (
    df_vcf.iloc[:, 1:].apply(lambda x: x.eq(2) | x.eq(2.0)).sum(axis=1) * 2
) + (
    df_vcf.iloc[:, 1:].apply(lambda x: x.eq(1) | x.eq(1.0)).sum(axis=1) * 1
)

df = df_vcf[["IID", "count_synonymous"]]
print(df.head())

,IID,count_synonymous
0,QD00012_0001,373
1,QD00015_0001,561
2,QD00019_0001,422
3,QD00021_0001,426
4,QD00027_0001,487


In [19]:
df1=pd.read_csv('./Phenotype.tsv', sep='\t') #Here is an example of the phenotype file
pheno = df1.loc[:,['FID', 'IID', 'Phenotype']]
display(pheno.head())

,FID,IID,Phenotype
0,QD001,QD001,2
1,QD00199,QD00199,5
2,QD0001,QD0001,7
3,QD0000012,QD0000012,9
4,QD000001,QD000001,1


In [20]:
df_batch1=pd.read_csv('./study_pca.eigenvec', sep='\t') #Here is an example of the PCA file
df_batch1=df_batch1.loc[:,['IID','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10']]
display(df_batch1.head())

,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,QD001,-0.021001,0.062841,-0.046953,-0.011444,-0.012047,-0.006186,-0.010486,0.017712,-0.003843,0.015144
1,QD00199,-0.042598,0.106185,-0.031031,-0.025781,-0.016505,-0.002572,-0.040566,0.006609,0.043955,0.006157
2,QD0001,-0.031388,0.093170,-0.003714,-0.031037,-0.013036,0.018531,0.006814,-0.098985,-0.034902,0.013714
3,QD0000012,-0.030734,0.093848,0.005103,-0.039199,-0.008006,0.023067,-0.006914,-0.048694,-0.016291,0.009329
4,QD000001,-0.031798,0.087201,-0.020245,-0.027268,-0.012689,-0.006302,-0.034480,0.025765,0.019539,0.013497


In [21]:
#df1 also has covariate columns in addition to phenotype here
df_batch1_merge = pd.merge(df_batch1, df1, on=['IID'], how='left') #Merge the PCA with your covariate file
#Here Age, sex, steroid.medication, hx.glaucoma are covariates.
df_batch1_merge=df_batch1_merge.loc[:,['FID','IID','Age','Sex','steroid.medication','hx.glaucoma','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10']]
#Then merge your dataframe with covariates with the dataframe with the no. of synonymous mutations
df_batch1_merge=pd.merge(df_batch1_merge, df_synonymous_summary, on=['IID'], how='left')
display(df_batch1_merge)


,FID,IID,Age,Sex,steroid.medication,hx.glaucoma,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,count_synonymous
0,QD001,QD001,69,1,2,2,-0.021001,0.062841,-0.046953,-0.011444,-0.012047,-0.006186,-0.010486,0.017712,-0.003843,0.015144,16
1,QD00199,QD00199,60,1,2,2,-0.042598,0.106185,-0.031031,-0.025781,-0.016505,-0.002572,-0.040566,0.006609,0.043955,0.006157,38
2,QD0001,QD0001,61,1,2,2,-0.031388,0.093170,-0.003714,-0.031037,-0.013036,0.018531,0.006814,-0.098985,-0.034902,0.013714,123
3,QD0000012,QD0000012,60,1,2,2,-0.030734,0.093848,0.005103,-0.039199,-0.008006,0.023067,-0.006914,-0.048694,-0.016291,0.009329,50
4,QD000001,QD000001,79,1,2,2,-0.031798,0.087201,-0.020245,-0.027268,-0.012689,-0.006302,-0.034480,0.025765,0.019539,0.013497,47


In [11]:
df_batch1_merge.to_csv('my_covariate_file.csv', sep='\t', index=None)

In [13]:
pheno.to_csv("my_phenoFile.csv", sep='\t', index=None)